In [207]:
import os
import sys
# Locate the script in UnifiedInflation/examples and add to path one folder before, that is, UnifiedInflation/
# in order to be able to import quantuminflation
# ! Note: I found online that "__file__" sometimes can be problematic, So I'm using the solution provided in
# https://stackoverflow.com/questions/2632199/how-do-i-get-the-path-of-the-current-executed-file-in-python?lq=1
from inspect import getsourcefile
from os.path import abspath
from tabnanny import verbose

#cws = abspath(getsourcefile(lambda:0))
cws = os.getcwd()
cws = os.sep.join(cws.split(os.sep)[:-1])  # Remove the script filename to get the directory of the script
cws = cws + os.sep + os.pardir  # Go one folder above UnifiedInflation/examples -> UnifiedInflation/
sys.path.append(cws)
###############################################################################

import itertools
import pickle

import numpy as np
from quantuminflation.InflationProblem import InflationProblem
from quantuminflation.InflationSDP import InflationSDP
from quantuminflation.useful_distributions import *
from quantuminflation.general_tools import to_numbers

from quantuminflation.fast_npa import calculate_momentmatrix
import itertools

from ncpol2sdpa.nc_utils import apply_substitutions
import sympy as sp

Calculate the objective function. First we encode $U_{0|0}$, $U_{0|1}$ and $V$. From the paper:

$\begin{aligned} U_{0 \mid 0}(p)=& 1-p_{A}(0 \mid 1)-p_{B}(0 \mid 1)+2 p(00 \mid 11) \\ U_{0 \mid 1}(p)=& 1-0.0329 p_{B}(0 \mid 0)-0.7117 p_{B}(0 \mid 2) \\ &-0.0329 p_{A}(0 \mid 0)-0.8418 p(0,0 \mid 0,0) \\ &+0.6359 p(0,0 \mid 0,2)-0.7117 p_{A}(0 \mid 2) \\ &+0.6359 p(0,0 \mid 2,0)+0.4360 p(0,0 \mid 2,2) \end{aligned}$

$\begin{aligned} V(p)=& 0.1590+0.8372 p_{B}(0 \mid 0)+0.0031 p_{B}(0 \mid 1) \\ &-0.1544 p_{A}(0 \mid 0)-0.6132 p(0,0 \mid 0,0) \\ &+0.5547 p(0,0 \mid 0,1)+0.5884 p_{A}(0 \mid 1) \\ &-0.5902 p(0,0 \mid 1,0)-0.7404 p(0,0 \mid 11) \end{aligned}$

In [208]:
# The U first

### The first input
outs_cardinality_ef = [2, 2]
ins_cardinality_ef  = [3, 3]
number_alpha = 2
number_eta = 2
U = np.zeros((number_alpha, number_eta, *outs_cardinality_ef, *ins_cardinality_ef), dtype=np.longdouble)

U[0,0,:,:,0,0] += 1
U[0,0,0,:,1,0] += -1
U[0,0,:,0,0,1] += -1
U[0,0,0,0,1,1] += 2

U[1,0,:,:,0,0] += 1
U[1,0,:,:,:,:] -= U[0,0,:,:,:,:]

U_01 = np.zeros((*outs_cardinality_ef,*ins_cardinality_ef))
U[0,1,:,:,0,0] += 1

U[0,1,:,0,0,0] += -0.0329
U[0,1,:,0,0,2] += -0.7117

U[0,1,0,:,0,0] += -0.0329
U[0,1,0,:,2,0] += -0.7117

U[0,1,0,0,0,0] += -0.8418
U[0,1,0,0,0,2] += +0.6359
U[0,1,0,0,2,0] += +0.6359
U[0,1,0,0,2,2] += +0.4360

U[1,1,:,:,0,0] += 1
U[1,1,:,:,:,:] -= U[0,1,:,:,:,:]

# The V
outs_cardinality_ug = [U.shape[0], 2]
ins_cardinality_ug  = [U.shape[1], 2]

V = np.zeros((*outs_cardinality_ug,*ins_cardinality_ug), dtype=np.longdouble)  # V[alpha,c,eta,z]
V[:,:,0,0] += +0.1590

V[0,:,0,0] += -0.1544
V[0,:,1,0] += +0.5884

V[:,0,0,0] += +0.8372
V[:,0,0,1] += +0.0031
V[0,0,0,0] += -0.6132
V[0,0,0,1] += +0.5547
V[0,0,1,0] += -0.5902
V[0,0,1,1] += -0.7404

Do the index contraction for the W

In [209]:
#W = np.einsum('AcEz,AEabxy->abcxyz', V2[:-1,:,:,:], U)
W = np.einsum('AcEz,AEabxy->abcxyz', V[:,:,:,:], U)

Define an auxiliary scenario. We need to run a trivial problem to compile the functions. This will be improved later.

In [210]:
InfProb = InflationProblem( hypergraph=[[1, 1]],outcomes_per_party=[2, 2],settings_per_party=[1, 1],inflation_level_per_source=[1])
print('Currently needed to run a trivial problem to precompile the functions, this can be changed later')
InfSDP = InflationSDP(InfProb, verbose=0)
InfSDP._build_columns([[],[0]])
cols_small = [np.array(col, dtype=np.uint8) for col in InfSDP.generating_monomials]
momentmatrix, vardic = calculate_momentmatrix(cols_small, np.array(InfSDP.names))

Currently needed to run a trivial problem to precompile the functions, this can be changed later


Calculating moment matrix    : 100%|██████████| 2/2 [00:00<?, ?it/s]


# Testing

Check if $U_{0|0}$, $U_{0|1}$ and $V$ are valid NBF over AQ. We'll try to find the maximum and minimum over AQ.

First $U_{0|0}$, $U_{0|1}$

!! After checking the first time, we renormalize to machine precision.

In [211]:
InfProb = InflationProblem( hypergraph=[[1, 1]],
                            outcomes_per_party=outs_cardinality_ef,
                            settings_per_party=ins_cardinality_ef,
                            inflation_level_per_source=[1],
                            names=['A', 'B'])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
filename_label = 'AQ_U'
InfSDP.generate_relaxation( column_specification='local1',
                            max_monomial_length=False,
                            find_physical_monomials=False,
                            use_linear_constraints=False,
                            sandwich_positivity=False,
                            load_from_file=False,
                            filename_label=filename_label)
meas = InfSDP.measurements
for eta, alpha in itertools.product(range(2), repeat=2):
    obj = 0
    for a,b,x,y in itertools.product(*[range(i) for i in U.shape[2:]]):
        Pi_A = meas[0][0][x][a] if a == 0 else sp.S.One - meas[0][0][x][0]
        Pi_B = meas[1][0][y][b] if b == 0 else sp.S.One - meas[1][0][y][0]
        obj += U[alpha,eta,a,b,x,y]*Pi_A*Pi_B
    # if (alpha, eta) == (0, 1):
    #     obj -= 0.8

    obj = sp.expand(obj)
    print("U_"+str(alpha)+str(eta)+" =", obj)
    InfSDP.set_objective(objective=-obj)
    InfSDP.solve()  # By default it maximizes
    minimo = InfSDP.primal_objective
    print("minimum over AQ of U_"+str(alpha)+str(eta), -InfSDP.primal_objective)
    InfSDP.set_objective(objective=obj)
    InfSDP.solve()  # By default it maximizes
    print("Maximum over AQ of U_"+str(alpha)+str(eta), InfSDP.primal_objective)
    maximo = InfSDP.primal_objective
    # Add the minimum and divide by the difference between the maximum and minimum
    # U[alpha,eta,:,:,0,0] += minimo
    # U[alpha,eta,:,:,:,:] /= maximo - minimo

# obj = (1 +
#         -0.0329*meas[1][0][0][0] +
#         -0.7117*meas[1][0][2][0] +
#         -0.0329*meas[0][0][0][0] +
#         -0.7117*meas[0][0][2][0] +
#         -0.8418*meas[0][0][0][0]*meas[1][0][0][0] +
#         +0.6359*meas[0][0][0][0]*meas[1][0][2][0] +
#         +0.6359*meas[0][0][2][0]*meas[1][0][0][0] +
#         +0.4360*meas[0][0][2][0]*meas[1][0][2][0] )
# obj = sp.expand(obj)
# InfSDP.set_objective(objective=-obj)
# print(obj)
# InfSDP.solve()  # By default it maximizes
# print("Minimum over AQ of V", InfSDP.primal_objective)
# InfSDP.set_objective(objective=obj)
# InfSDP.solve()  # By default it maximizes
# print("Maximum over AQ of V", InfSDP.primal_objective)

Reassigning moment matrix indices: 100%|██████████| 16/16 [00:00<00:00, 16012.61it/s]


U_00 = 1.0 - 1.0*A_1_1_0 + 2.0*A_1_1_0*B_1_1_0 - 1.0*B_1_1_0
minimum over AQ of U_00 -1.4420064919917763e-08
Maximum over AQ of U_00 1.0000000128535476
U_10 = 1.0*A_1_1_0 - 2.0*A_1_1_0*B_1_1_0 + 1.0*B_1_1_0
minimum over AQ of U_10 -1.2853547667126446e-08
Maximum over AQ of U_10 1.000000014420065
U_01 = 1.0 - 0.0329*A_1_0_0 - 0.8418*A_1_0_0*B_1_0_0 + 0.6359*A_1_0_0*B_1_2_0 - 0.7117*A_1_2_0 + 0.6359*A_1_2_0*B_1_0_0 + 0.436*A_1_2_0*B_1_2_0 - 0.0329*B_1_0_0 - 0.7117*B_1_2_0
minimum over AQ of U_01 1.5576856671417616e-05
Maximum over AQ of U_01 1.0000207289194862
U_11 = 0.0329*A_1_0_0 + 0.8418*A_1_0_0*B_1_0_0 - 0.6359*A_1_0_0*B_1_2_0 + 0.7117*A_1_2_0 - 0.6359*A_1_2_0*B_1_0_0 - 0.436*A_1_2_0*B_1_2_0 + 0.0329*B_1_0_0 + 0.7117*B_1_2_0
minimum over AQ of U_11 -2.0728919393495328e-05
Maximum over AQ of U_11 0.9999844231346401


Now lets check $V$

In [212]:
InfProb = InflationProblem( hypergraph=[[1, 1]],
                            outcomes_per_party=outs_cardinality_ug,
                            settings_per_party=ins_cardinality_ug,
                            inflation_level_per_source=[1],
                            names=['A', 'B'])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
filename_label = 'AQ_V'
InfSDP.generate_relaxation( column_specification='local1',
                            max_monomial_length=False,
                            find_physical_monomials=False,
                            use_linear_constraints=False,
                            sandwich_positivity=False,
                            load_from_file=False,
                            filename_label=filename_label)
meas = InfSDP.measurements

obj = 0
for a,b,x,y in itertools.product(*[range(i) for i in V.shape]):
    Pi_A = meas[0][0][x][a] if a == 0 else 1-meas[0][0][x][0]
    Pi_B = meas[1][0][y][b] if b == 0 else 1-meas[1][0][y][0]
    obj += V[a,b,x,y]*Pi_A*Pi_B
obj = sp.expand(obj)
print(obj)
InfSDP.set_objective(objective=-obj)
InfSDP.solve()  # By default it maximizes
print("minimum over AQ of V_", -InfSDP.primal_objective)
minimo = InfSDP.primal_objective
InfSDP.set_objective(objective=obj)
InfSDP.solve()  # By default it maximizes
print("Maximum over AQ of V_", InfSDP.primal_objective)
maximo = InfSDP.primal_objective
# V[:,:,0,0] += minimo
# V[:,:,:,:] /= maximo - minimo


obj1 = (0.1590 +
        +0.8372*meas[1][0][0][0] +
        +0.0031*meas[1][0][1][0] +
        -0.1544*meas[0][0][0][0] +
        +0.5884*meas[0][0][1][0] +
        -0.6132*meas[0][0][0][0]*meas[1][0][0][0] +
        +0.5547*meas[0][0][0][0]*meas[1][0][1][0] +
        -0.5902*meas[0][0][1][0]*meas[1][0][0][0] +
        -0.7404*meas[0][0][1][0]*meas[1][0][1][0]  ) 
print(obj1-obj)
# print(obj)
# InfSDP.set_objective(objective=-obj)
# InfSDP.solve()  # By default it maximizes
# print("minimum over AQ of V", InfSDP.primal_objective)
# InfSDP.set_objective(objective=obj)
# InfSDP.solve()  # By default it maximizes
# print("Maximum over AQ of V", InfSDP.primal_objective)

Reassigning moment matrix indices: 100%|██████████| 9/9 [00:00<?, ?it/s]

0.159 - 0.1544*A_1_0_0 - 0.6132*A_1_0_0*B_1_0_0 + 0.5547*A_1_0_0*B_1_1_0 + 0.5884*A_1_1_0 - 0.5902*A_1_1_0*B_1_0_0 - 0.7404*A_1_1_0*B_1_1_0 + 0.8372*B_1_0_0 + 0.0031*B_1_1_0
minimum over AQ of V_ -2.5077128465444964e-06
Maximum over AQ of V_ 0.9999673614230343
0


## Check CHSH with the SDP

In [213]:
InfProb = InflationProblem(hypergraph=[[1, 1]], outcomes_per_party=[2,2], settings_per_party=[2,2], inflation_level_per_source=[1],names=['A', 'B'])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
InfSDP.generate_relaxation( column_specification='local1',
                            max_monomial_length=False,
                            find_physical_monomials=False,
                            use_linear_constraints=False,
                            sandwich_positivity=False,
                            load_from_file=False,
                            filename_label=filename_label)
meas = InfSDP.measurements
A0, A1, B0, B1 = 1-2*meas[0][0][0][0], 1-2*meas[0][0][1][0], 1-2*meas[1][0][0][0], 1-2*meas[1][0][1][0]
obj =  A0*B0+A0*B1+A1*B0-A1*B1
obj = sp.expand(obj)
print(obj)
InfSDP.set_objective(objective=+obj)
InfSDP.solve(interpreter='CVXPY')  # By default it maximizes
print("minimum over AQ of CHSH", InfSDP.primal_objective)

Reassigning moment matrix indices: 100%|██████████| 9/9 [00:00<?, ?it/s]

2 - 4*A_1_0_0 + 4*A_1_0_0*B_1_0_0 + 4*A_1_0_0*B_1_1_0 + 4*A_1_1_0*B_1_0_0 - 4*A_1_1_0*B_1_1_0 - 4*B_1_0_0
minimum over AQ of CHSH 2.8284271790023405


Do the index contraction for the W

In [214]:
#W = np.einsum('AcEz,AEabxy->abcxyz', V2[:-1,:,:,:], U)
W = np.einsum('AcEz,AEabxy->abcxyz', V[:,:,:,:], U)

Define an auxiliary scenario. We need to run a trivial problem to compile the functions. This will be improved later.

# Functions for Collins Gisin notation

In [215]:
from quantuminflation.general_tools import find_permutation, apply_substitutions
from ncpol2sdpa.nc_utils import flatten

### From probabilitiy to CG


In [216]:
def prob_to_CG_2parties(prob, outcomes_per_party, settings_per_party):
    dims = np.multiply(np.array(outcomes_per_party)-1, np.array(settings_per_party)) + 1
    p_CG = np.zeros(dims)
    for a, b in itertools.product(*[range(d) for d in dims]):
        if a == 0 and b == 0:
            p_CG[0, 0] = 1
        if a != 0 and b == 0:
            a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
            p_CG[a, 0] = np.sum(prob[a_,:,x,0])
        if a == 0 and b != 0:
            b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
            p_CG[0, b] = np.sum(prob[:,b_,0,y])
        if a != 0 and b != 0:
            a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
            b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
            p_CG[a, b] = prob[a_,b_,x,y]
    return p_CG

def prob_to_CG_3parties(prob, outcomes_per_party, settings_per_party):
    dims = np.multiply(np.array(outcomes_per_party)-1, np.array(settings_per_party)) + 1
    p_CG = np.zeros(dims)
    for a, b, c in itertools.product(*[range(d) for d in dims]):
        if a == 0 and b == 0 and c == 0:
            p_CG[0, 0, 0] = 1
        if a != 0 and b == 0 and c == 0:
            a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
            p_CG[a, 0, 0] = np.sum(prob[a_,:,:,x,0,0])
        if a == 0 and b != 0 and c == 0:
            b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
            p_CG[0, b, 0] = np.sum(prob[:,b_,:,0,y,0])
        if a == 0 and b == 0 and c != 0:
            c_, z = np.unravel_index(c-1, (outcomes_per_party[2]-1,settings_per_party[2]))
            p_CG[0, 0, c] = np.sum(prob[:,:,c_,0,0,z])
        if a != 0 and b != 0 and c == 0:
            a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
            b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
            p_CG[a, b, 0] = np.sum(prob[a_,b_,:,x,y,0])
        if a != 0 and b == 0 and c != 0:
            a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
            c_, z = np.unravel_index(c-1, (outcomes_per_party[2]-1,settings_per_party[2]))
            p_CG[a, 0, c] = np.sum(prob[a_,:,c_,x,0,z])
        if a == 0 and b != 0 and c != 0:
            b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
            c_, z = np.unravel_index(c-1, (outcomes_per_party[2]-1,settings_per_party[2]))
            p_CG[0, b, c] = np.sum(prob[:,b_,c_,0,y,z])
        if a != 0 and b != 0 and c != 0:
            a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
            b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
            c_, z = np.unravel_index(c-1, (outcomes_per_party[2]-1,settings_per_party[2]))
            p_CG[a, b, c] = prob[a_,b_,c_,x,y,z]
    return p_CG

# outcomes_per_party = [2, 2, 2]
# settings_per_party = [2, 2, 2]
# prob = np.ones([2,2,2,2,2,2])/8
# prob_to_CG_2parties(prob, outcomes_per_party, settings_per_party)

# outcomes_per_party = [2, 2]
# settings_per_party = [2, 2]
# prob = np.ones([2,2,2,2])/4
# prob_to_CG_2parties(prob, outcomes_per_party, settings_per_party)

### From CG to probability

In [311]:
def CG_to_prob_2party(p_CG, outcomes_per_party, settings_per_party):
    prob = np.zeros((*outcomes_per_party, *settings_per_party))
    for a, b, x, y in itertools.product(*[range(i) for i in (*outcomes_per_party, *settings_per_party)]):
        if a != outcomes_per_party[0]-1 and b != outcomes_per_party[1]-1:
            # An alternative to ravel_multi_index is to just output outcomes_per_party[0]*a+x and outcomes_per_party[1]*b+y, which is faster,
            # but in my mind this code is more flexible. If performance is an issue when converting
            # from Collins Gisin to normal probability, the straight multiplication should be used as it is is 1000x faster.
            # But I don't forsee this being a bottleneck.
            ax = np.ravel_multi_index([a,x],[outcomes_per_party[0],settings_per_party[0]])
            by = np.ravel_multi_index([b,y],[outcomes_per_party[1],settings_per_party[1]])
            prob[a, b, x, y] = p_CG[1+ax, 1+by]

        if a == outcomes_per_party[0]-1 and b != outcomes_per_party[1]-1:
            by = np.ravel_multi_index([b,y],[outcomes_per_party[1],settings_per_party[1]])
            suma = 0
            for a_ in range(outcomes_per_party[0]-1):
                ax = np.ravel_multi_index([a_,x],[outcomes_per_party[0],settings_per_party[0]])
                suma += p_CG[1+ax, 1+by]
            prob[a, b, x, y] = p_CG[0, 1+by] - suma

        if a != outcomes_per_party[0]-1 and b == outcomes_per_party[1]-1:
            ax = np.ravel_multi_index([a,x],[outcomes_per_party[0],settings_per_party[0]])
            suma = 0
            for b_ in range(outcomes_per_party[1]-1):
                by = np.ravel_multi_index([b_,y],[outcomes_per_party[1],settings_per_party[1]])
                suma += p_CG[1+ax, 1+by]
            prob[a, b, x, y] = p_CG[1+ax, 0] - suma

        if a == outcomes_per_party[0]-1 and b == outcomes_per_party[1]-1:
            suma_A = 0
            for a_ in range(outcomes_per_party[0]-1):
                ax = np.ravel_multi_index([a_,x],[outcomes_per_party[0],settings_per_party[0]])
                suma_A += p_CG[1+ax, 0]

            suma_B = 0
            for b_ in range(outcomes_per_party[1]-1):
                by = np.ravel_multi_index([b_,y],[outcomes_per_party[1],settings_per_party[1]])
                suma_B += p_CG[0, 1+by]

            suma_AB = 0
            for a_ in range(outcomes_per_party[0]-1):
                for b_ in range(outcomes_per_party[1]-1):
                    ax = np.ravel_multi_index([a_,x],[outcomes_per_party[0],settings_per_party[0]])
                    by = np.ravel_multi_index([b_,y],[outcomes_per_party[1],settings_per_party[1]])
                    suma_AB += p_CG[1+ax, 1+by]

            prob[a, b, x, y] = 1 - suma_A - suma_B + suma_AB
    #assert np.allclose(prob.sum(axis=(0,1)), 1)
    return prob

In [312]:
def CG_to_prob_3party(p_CG, outcomes_per_party, settings_per_party):
    prob = np.zeros((*outcomes_per_party, *settings_per_party))
    a_last, b_last, c_last = outcomes_per_party[0]-1, outcomes_per_party[1]-1, outcomes_per_party[2]-1
    for a, b, c, x, y, z in itertools.product(*[range(i) for i in (*outcomes_per_party, *settings_per_party)]):
        ax = a*outcomes_per_party[0] + x #np.ravel_multi_index([a,x],[outcomes_per_party[0],settings_per_party[0]])
        by = b*outcomes_per_party[1] + y #np.ravel_multi_index([b,y],[outcomes_per_party[1],settings_per_party[1]])
        cz = c*outcomes_per_party[2] + z #np.ravel_multi_index([c,z],[outcomes_per_party[2],settings_per_party[2]])
        if a != a_last and b != b_last and c != c_last:
            prob[a, b, c, x, y, z] = p_CG[1+ax, 1+by, 1+cz]

        if a == a_last and b != b_last and c != c_last:
            suma = sum([p_CG[1+a_*outcomes_per_party[0]+x, 1+by, 1+cz] for a_ in range(a_last)])  # Sum over all a_ except the last
            prob[a, b, c, x, y, z] = p_CG[0, 1+by, 1+cz] - suma

        if a != a_last and b == b_last and c != c_last:
            suma = sum([p_CG[1+ax, 1+b_*outcomes_per_party[1]+y, 1+cz] for b_ in range(b_last)])  
            prob[a, b, c, x, y, z] = p_CG[1+ax, 0, 1+cz] - suma

        if a != a_last and b != b_last and c == c_last:
            suma = sum([p_CG[1+ax, 1+by, 1+c_*outcomes_per_party[2]+z] for c_ in range(c_last)])  
            prob[a, b, c, x, y, z] = p_CG[1+ax, 1+by, 0] - suma
        
        if a == a_last and b == b_last and c != c_last:
            suma_A  = sum([p_CG[1+a_*outcomes_per_party[0]+x, 0, 1+cz] for a_ in range(a_last)]) 
            suma_B  = sum([p_CG[0, 1+b_*outcomes_per_party[1]+y, 1+cz] for b_ in range(b_last)]) 
            suma_AB = sum([p_CG[1+a_*outcomes_per_party[0]+x, 1+b_*outcomes_per_party[1]+y, 1+cz] for b_ in range(b_last) for a_ in range(a_last)])
            prob[a, b, c, x, y, z] = p_CG[0, 0, 1+cz] - suma_A - suma_B + suma_AB

        if a == a_last and b != b_last and c == c_last:
            suma_A  = sum([p_CG[1+a_*outcomes_per_party[0]+x, 1+by, 0] for a_ in range(a_last)]) 
            suma_C  = sum([p_CG[0, 1+by, 1+c_*outcomes_per_party[2]+z] for c_ in range(c_last)]) 
            suma_AC = sum([p_CG[1+a_*outcomes_per_party[0]+x, 1+by, 1+c_*outcomes_per_party[2]+z] for a_ in range(a_last) for c_ in range(c_last)])
            prob[a, b, c, x, y, z] = p_CG[0, 1+by, 0] - suma_A - suma_C + suma_AC

        if a != a_last and b == b_last and c == c_last:
            suma_B  = sum([p_CG[1+ax, 1+b_*outcomes_per_party[1]+y, 0] for b_ in range(b_last)]) 
            suma_C  = sum([p_CG[1+ax, 0, 1+c_*outcomes_per_party[2]+z] for c_ in range(c_last)]) 
            suma_BC = sum([p_CG[1+ax, 1+b_*outcomes_per_party[1]+y, 1+c_*outcomes_per_party[2]+z] for b_ in range(b_last) for c_ in range(c_last)])
            prob[a, b, c, x, y, z] = p_CG[1+ax, 0, 0] - suma_B - suma_C + suma_BC

        if a == a_last and b == b_last and c == c_last:
            suma_A   = sum([p_CG[1+a_*outcomes_per_party[0]+x, 0, 0] for a_ in range(a_last)]) 
            suma_B   = sum([p_CG[0, 1+b_*outcomes_per_party[1]+y, 0] for b_ in range(b_last)]) 
            suma_C   = sum([p_CG[0, 0, 1+c_*outcomes_per_party[2]+z] for c_ in range(c_last)]) 
            suma_AB  = sum([p_CG[1+a_*outcomes_per_party[0]+x, 1+b_*outcomes_per_party[1]+y, 0] for b_ in range(b_last) for a_ in range(a_last)])
            suma_AC  = sum([p_CG[1+a_*outcomes_per_party[0]+x, 0, 1+c_*outcomes_per_party[2]+z] for a_ in range(a_last) for c_ in range(c_last)])
            suma_BC  = sum([p_CG[0, 1+b_*outcomes_per_party[1]+y, 1+c_*outcomes_per_party[2]+z] for b_ in range(b_last) for c_ in range(c_last)])
            suma_ABC = sum([p_CG[1+a_*outcomes_per_party[0]+x, 1+b_*outcomes_per_party[1]+y, 1+c_*outcomes_per_party[2]+z] for a_ in range(a_last) for b_ in range(b_last) for c_ in range(c_last)])
            prob[a, b, c, x, y, z] = 1 - suma_A - suma_B - suma_C + suma_AB + suma_AC + suma_BC - suma_ABC

    assert np.allclose(prob.sum(axis=(0,1,2)), 1)
    return prob

In [334]:
def extract_p_CG_from_AQ_SDP(momentmatrix_solution, measurements, columns, outcomes_per_party, settings_per_party):
    p_CG_0 = momentmatrix_solution[0,:]
    dims = np.multiply(np.array(outcomes_per_party)-1, np.array(settings_per_party)) + 1
    nr_parties = len(outcomes_per_party)
    if nr_parties == 2:
        correct_order = np.zeros(dims, dtype=object)
        for a,b in itertools.product(*[range(i) for i in dims]):
            prod = 1
            if a != 0:
                a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
                prod *= measurements[0][0][x][a_]
            if b != 0:
                b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
                prod *= measurements[1][0][y][b_]
            correct_order[a,b] = prod
    elif nr_parties == 3:
        correct_order = np.zeros(dims, dtype=object)
        for a,b,c in itertools.product(*[range(i) for i in dims]):
            prod = 1
            if a != 0:
                a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
                prod *= measurements[0][0][x][a_]
            if b != 0:
                b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
                prod *= measurements[1][0][y][b_]
            if c != 0:
                c_, z = np.unravel_index(c-1, (outcomes_per_party[2]-1,settings_per_party[2]))
                prod *= measurements[2][0][z][c_]
            correct_order[a,b,c] = prod

    perm = find_permutation(columns, correct_order.flatten())
    p_CG_permuted = np.array([p_CG_0[perm[i]] for i in range(len(p_CG_0))])
    p_CG = p_CG_permuted.reshape(correct_order.shape)
    return p_CG

def p_CG_in_SDP_vector_AQ_SDP(p_CG_in, measurements, columns, outcomes_per_party, settings_per_party):
    p_CG_in = p_CG_in.flatten()
    dims = np.multiply(np.array(outcomes_per_party)-1, np.array(settings_per_party)) + 1
    nr_parties = len(outcomes_per_party)
    if nr_parties == 2:
        correct_order = np.zeros(dims, dtype=object)
        for a,b in itertools.product(*[range(i) for i in dims]):
            prod = 1
            if a != 0:
                a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
                prod *= measurements[0][0][x][a_]
            if b != 0:
                b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
                prod *= measurements[1][0][y][b_]
            correct_order[a,b] = prod
    elif nr_parties == 3:
        correct_order = np.zeros(dims, dtype=object)
        for a,b,c in itertools.product(*[range(i) for i in dims]):
            prod = 1
            if a != 0:
                a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
                prod *= measurements[0][0][x][a_]
            if b != 0:
                b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
                prod *= measurements[1][0][y][b_]
            if c != 0:
                c_, z = np.unravel_index(c-1, (outcomes_per_party[2]-1,settings_per_party[2]))
                prod *= measurements[2][0][z][c_]
            correct_order[a,b,c] = prod

    perm = find_permutation( correct_order.flatten(), columns)
    p_CG_permuted = np.array([p_CG_in[perm[i]] for i in range(len(p_CG_in))])
    #p_CG = p_CG_permuted.reshape(correct_order.shape)
    return p_CG_permuted

In [219]:
outcomes_per_party = [2, 2]
settings_per_party = [2, 2]
InfProb = InflationProblem(hypergraph=[[1, 1]], 
                             outcomes_per_party=outcomes_per_party,
                             settings_per_party=settings_per_party,
                             inflation_level_per_source=[1], names=['A', 'B'])
InfSDP = InflationSDP(InfProb, commuting=True, verbose=0)
InfSDP.generate_relaxation( column_specification='local1',
                            max_monomial_length=False,
                            find_physical_monomials=False,
                            use_linear_constraints=False,
                            sandwich_positivity=False,
                            load_from_file=False,
                            filename_label=filename_label)
meas = InfSDP.measurements
A0, A1, B0, B1 = 1-2*meas[0][0][0][0], 1-2*meas[0][0][1][0], 1-2*meas[1][0][0][0], 1-2*meas[1][0][1][0]
obj =  A0*B0+A0*B1+A1*B0-A1*B1
obj = sp.expand(obj)
print(obj)
InfSDP.set_objective(objective=+obj)
InfSDP.solve(interpreter='MOSEKFusion', solverparameters={'solve_dual':True})  # By default it maximizes
print("minimum over AQ of CHSH", InfSDP.primal_objective)


Gamma = InfSDP.solution_object['G']
generating_set = InfSDP.generating_monomials
p_CG_0 = Gamma[0,:]
#CG_shape = [3*1+1,3*1+1,2*1+1]
#p_CG = p_CG.reshape(CG_shape)  # In collins gisin notaton

meas = InfSDP.measurements
dims = np.multiply(np.array(outcomes_per_party)-1, np.array(settings_per_party)) + 1
correct_order = np.zeros(dims, dtype=object)
for a,b in itertools.product(*[range(i) for i in dims]):
    prod = 1
    if a != 0:
        a_, x = np.unravel_index(a-1, (outcomes_per_party[0]-1,settings_per_party[0]))
        prod *= meas[0][0][x][a_]
    if b != 0:
        b_, y = np.unravel_index(b-1, (outcomes_per_party[1]-1,settings_per_party[1]))
        prod *= meas[1][0][y][b_]
    correct_order[a,b] = prod

perm = find_permutation(InfSDP.generating_monomials_sym, correct_order.flatten())
p_CG_permuted = np.array([p_CG_0[perm[i]] for i in range(len(p_CG_0))])
p_CG = p_CG_permuted.reshape(correct_order.shape)
# p_CG[0,0]
# p_CG_0, p_CG_permuted, InfSDP.generating_monomials_sym
p_CG, correct_order

Reassigning moment matrix indices: 100%|██████████| 9/9 [00:00<00:00, 9007.10it/s]

4*A_1_0_0*B_1_0_0 + 4*A_1_0_0*B_1_1_0 - 4*A_1_0_0 + 4*A_1_1_0*B_1_0_0 - 4*A_1_1_0*B_1_1_0 - 4*B_1_0_0 + 2
minimum over AQ of CHSH 2.0000000484582166


(array([[1.        , 0.45298065, 0.47846019],
        [0.45295239, 0.32969124, 0.33501134],
        [0.47845363, 0.33500597, 0.09377549]]),
 array([[1, B_1_0_0, B_1_1_0],
        [A_1_0_0, A_1_0_0*B_1_0_0, A_1_0_0*B_1_1_0],
        [A_1_1_0, A_1_1_0*B_1_0_0, A_1_1_0*B_1_1_0]], dtype=object))

# Now we do the real scenario

In [220]:
from quantuminflation.general_tools import find_permutation, apply_substitutions
from ncpol2sdpa.nc_utils import flatten
import cvxpy as cp

In [224]:
InfProb = InflationProblem( hypergraph=[[1, 1, 1]],
                            outcomes_per_party=W.shape[:3],
                            settings_per_party=W.shape[3:],
                            inflation_level_per_source=[1],
                            names=['A', 'B', 'C'])

InfSDP = InflationSDP(InfProb, commuting=False, verbose=2)
filename_label = 'AQ'
InfSDP.generate_relaxation( column_specification= [[], [0], [1], [2], [0, 1], [0, 2], [1, 2], [0, 1, 2]],
                            max_monomial_length=False,
                            find_physical_monomials=False,
                            use_linear_constraints=False,
                            sandwich_positivity=False,
                            load_from_file=False,
                            filename_label=filename_label)

# Going to calculate $W$ 'by hand' s.t. I avoid possible problems from using einsum.
# W = np.zeros(W.shape)
# for a,b,c,x,y,z in itertools.product(*[range(i) for i in W.shape]):
#     for alpha, eta in itertools.product(*[range(i) for i in (outs_cardinality_ug[0], ins_cardinality_ug[0])]):
#         W[a,b,c,x,y,z] += U[alpha,eta,a,b,x,y]*V[alpha,c,eta,z]

meas = InfSDP.measurements

obj = 0
for a,b,c,x,y,z in itertools.product(*[range(i) for i in W.shape]):
    #print(a,b,c,x,y,z)
    Pi_A = meas[0][0][x][0]
    if a == 1:
        Pi_A = sp.S.One - meas[0][0][x][0]
    Pi_B = meas[1][0][y][0]
    if b == 1:
        Pi_B = sp.S.One - meas[1][0][y][0]
    Pi_C = meas[2][0][z][0]
    if c == 1:
        Pi_C = sp.S.One - meas[2][0][z][0]
    #Pi_B = meas[1][0][y][b] if b == 0 else sp.S.One - meas[1][0][y][0]
    #Pi_C = meas[2][0][z][c] if c == 0 else sp.S.One - meas[2][0][z][0]
    obj += sp.expand(W[a,b,c,x,y,z]*Pi_A*Pi_B*Pi_C)

A0 = meas[0][0][0][0]
A1 = meas[0][0][1][0]
A2 = meas[0][0][2][0]

B0 = meas[1][0][0][0]
B1 = meas[1][0][1][0]
B2 = meas[1][0][2][0]

C0 = meas[2][0][0][0]
C1 = meas[2][0][1][0]

obj = sp.expand(obj)
InfSDP.use_linear_constraints = False
InfSDP.set_objective(objective=-obj)

InfSDP.solve(interpreter='MOSEKFusion', solverparameters={'solve_dual': False})  # By default it maximizes

#InfSDP.write_to_file('inflationMATLAB_'+filename_label+'.mat')

print(-InfSDP.primal_objective)

Gamma = InfSDP.solution_object['G']
generating_set = InfSDP.generating_monomials
p_CG = Gamma[0,:]
#CG_shape = [3*1+1,3*1+1,2*1+1]
#p_CG = p_CG.reshape(CG_shape)  # In collins gisin notaton

meas = InfSDP.measurements
correct_order = np.zeros(np.multiply(np.array(W.shape[:3])-1, np.array(W.shape[3:]))+1,dtype=object)
for a,b,c in itertools.product(*[range(i) for i in np.multiply(np.array(W.shape[:3])-1, np.array(W.shape[3:]))+1]):
    prod = 1
    if a != 0:
        a_, x = np.unravel_index(a-1, (W.shape[0],W.shape[3]))
        prod *= meas[0][0][x][a_]
    if b != 0:
        b_, y = np.unravel_index(b-1, (W.shape[1],W.shape[4]))
        prod *= meas[1][0][y][b_]
    if c != 0:
        c_, z = np.unravel_index(c-1, (W.shape[2],W.shape[5]))
        prod *= meas[2][0][z][c_]
    correct_order[a,b,c] = prod

perm = find_permutation(correct_order.flatten(), InfSDP.generating_monomials_sym)
p_CG_permuted = np.array([p_CG[perm[i]] for i in range(len(p_CG))])
p_CG = p_CG_permuted.reshape(correct_order.shape)
p_CG, correct_order

InflationProblem with [[1, 1, 1]] as hypergraph, (2, 2, 2) outcomes per party, (3, 3, 2) settings per party and [1] inflation copies per source.
Column structure: 1+A+B+C+AB+AC+BC+ABC
Number of columns: 48


Simplifying monomials        : 100%|██████████| 47/47 [00:00<00:00, 330.99it/s]


Number of known, semi-known and unknown variables = 47 0 226
Number of positive/physical unknown variables = 0


Reassigning moment matrix indices: 100%|██████████| 48/48 [00:00<00:00, 15997.35it/s]


Problem
  Name                   : InfSDP          
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1178            
  Cones                  : 0               
  Scalar variables       : 276             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.02            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   : InfSDP          
  Objective se

(array([[[1.        , 0.22684014, 0.09583572],
         [0.0479266 , 0.34813555, 0.34813556],
         [0.33313559, 0.4536352 , 0.50003946],
         [0.15548943, 0.09583573, 0.23391091]],
 
        [[0.23391089, 0.02622295, 0.06391691],
         [0.04149851, 0.04551215, 0.03627695],
         [0.27947417, 0.17408594, 0.12781529],
         [0.16136222, 0.06391692, 0.14029101]],
 
        [[0.4536352 , 0.50003944, 0.32268502],
         [0.46780652, 0.22684013, 0.15548943],
         [0.17408593, 0.13646818, 0.12781529],
         [0.16136222, 0.06187293, 0.07255239]],
 
        [[0.06187293, 0.07255239, 0.02024839],
         [0.04792661, 0.04551215, 0.05915016],
         [0.03094307, 0.03627695, 0.05915016],
         [0.03094306, 0.03354753, 0.04799816]]]),
 array([[[1, C_1_0_0, C_1_1_0],
         [B_1_0_0, B_1_0_0*C_1_0_0, B_1_0_0*C_1_1_0],
         [B_1_1_0, B_1_1_0*C_1_0_0, B_1_1_0*C_1_1_0],
         [B_1_2_0, B_1_2_0*C_1_0_0, B_1_2_0*C_1_1_0]],
 
        [[A_1_0_0, A_1_0_0*C_1_0_0, A_1

# Now we can find the U and V above with a seesaw

Get the SDP matrix structure for the AQ set for U and V

In [225]:
import scipy

First a function that optimizes over AQ probabilities given V and Us.

In [324]:
outcomes_per_party = [2, 2, 2]
settings_per_party = [3, 3, 2]
InfProblem = InflationProblem(  hypergraph=[[1, 1, 1]], outcomes_per_party=outcomes_per_party,
                                settings_per_party=settings_per_party, inflation_level_per_source=[1], names=['A', 'B', 'C'])
InfSDP_3p = InflationSDP(InfProblem, commuting=False, verbose=0)
InfSDP_3p.generate_relaxation( column_specification= 'local1', max_monomial_length=False, find_physical_monomials=False,
                            use_linear_constraints=False, sandwich_positivity=False,  load_from_file=False, use_numba=True)

def opt_over_AQ_p(InfSDP_3p, V, U):
    W = np.einsum('AcEz,AEabxy->abcxyz', V, U)
    outcomes_per_party = W.shape[:3]
    settings_per_party = W.shape[3:]
    meas = InfSDP_3p.measurements
    obj = 0
    for a,b,c,x,y,z in itertools.product(*[range(i) for i in W.shape]):
        Pi_A = meas[0][0][x][0]
        if a == 1:
            Pi_A = sp.S.One - meas[0][0][x][0]
        Pi_B = meas[1][0][y][0]
        if b == 1:
            Pi_B = sp.S.One - meas[1][0][y][0]
        Pi_C = meas[2][0][z][0]
        if c == 1:
            Pi_C = sp.S.One - meas[2][0][z][0]
        obj += sp.expand(W[a,b,c,x,y,z]*Pi_A*Pi_B*Pi_C)

    InfSDP_3p.set_objective(objective=-obj)  # By default it maximizes
    InfSDP_3p.solve(interpreter='MOSEKFusion', solverparameters={'solve_dual': True})  
    p_CG = extract_p_CG_from_AQ_SDP(InfSDP.solution_object['G'], meas, InfSDP_3p.generating_monomials_sym, outcomes_per_party, settings_per_party)
    #print(-InfSDP.primal_objective)
    return p_CG, InfSDP.solution_object['G'][0,:], -InfSDP.primal_objective

p_CG, p_CG_raw, optval = opt_over_AQ_p(InfSDP_3p, V, U)
#p_CG, optval
p = CG_to_prob_3party(p_CG, outcomes_per_party, settings_per_party)

Reassigning moment matrix indices: 100%|██████████| 48/48 [00:00<00:00, 48175.78it/s]


In [339]:
outcomes_per_party = [2, 2]
settings_per_party = [2, 2]
InfSDP_V = InflationSDP(InflationProblem(outcomes_per_party=outcomes_per_party,settings_per_party=settings_per_party))
InfSDP_V.generate_relaxation(column_specification='local1',load_from_file=False)

known_vars = [0, 1]
positionsmatrix = InfSDP_V.momentmatrix

# objective = {key: float(val) for key, val in objective.items()}

nr_variables = np.max(positionsmatrix) + 1
nr_known = len(known_vars)
nr_unknown = nr_variables - nr_known

Iden = scipy.sparse.eye(positionsmatrix.shape[0]) #cp.diag(np.ones(positionsmatrix.shape[0]))
F0 = scipy.sparse.lil_matrix(positionsmatrix.shape)
for variable in range(nr_known):
    F0[np.where(positionsmatrix == variable)] = known_vars[variable]
F_list = []
for var in range(nr_known, nr_variables):
    F = scipy.sparse.lil_matrix(positionsmatrix.shape)
    F[np.where(positionsmatrix == var)] = 1
    F_list.append(F)

#def opt_over_AQ_V(InfSDP_2p_V, U, p):
omega_alpha_c_eta_z = np.einsum('AEabxy,abcxyz->AcEz', U, p)
omega_alpha_c_eta_z_CG = prob_to_CG_2parties(omega_alpha_c_eta_z, outcomes_per_party, settings_per_party)
omega_alpha_c_eta_z_CG_raw = p_CG_in_SDP_vector_AQ_SDP(omega_alpha_c_eta_z_CG, InfSDP_V.measurements, InfSDP_V.generating_monomials_sym, outcomes_per_party, settings_per_party)

# meas = InfSDP_V.measurements
# obj = 0
# Vshape = [*outcomes_per_party, *settings_per_party]
# for alpha,c,eta,z in itertools.product(*[range(i) for i in Vshape]):
#     Pi_A = 1-sum([meas[0][0][eta][o_] for o_ in range(Vshape[0]-1)]) if alpha==(Vshape[0]-1) else meas[0][0][eta][alpha] 
#     Pi_C = 1-sum([meas[1][0][z  ][o_] for o_ in range(Vshape[1]-1)]) if c==(Vshape[1]-1) else meas[1][0][z][c] 
#     obj += sp.expand(omega_alpha_c_eta_z[alpha,c,eta,z]*Pi_A*Pi_C)
# obj = sp.expand(obj)
# InfSDP_V.set_objective(objective=obj)
# objective = InfSDP_V._objective_as_dict
# objective

Z = cp.Variable(positionsmatrix.shape, PSD=True)
Y = cp.Variable(positionsmatrix.shape, PSD=True)

dim = omega_alpha_c_eta_z_CG.size
v_i = cp.Variable(dim)

Fi_observable   = F_list[:dim]
Fi_unobservable = F_list[dim:]

constraints = [ *[cp.trace(F0 @ Z) == v_i[0]],
                *[cp.trace(Fi @ Z) == v_i[i]  for i, Fi in enumerate(Fi_observable)],
                *[cp.trace(Fi @ Z) == 0       for Fi in Fi_unobservable],
                *[cp.trace(F0 @ Y) == 1-v_i[0]],
                *[cp.trace(Fi @ Y) == -v_i[i] for i, Fi in enumerate(Fi_observable)],
                *[cp.trace(Fi @ Y) == 0       for Fi in Fi_unobservable]]

obj = v_i @ omega_alpha_c_eta_z_CG_raw

prob = cp.Problem(cp.Minimize(obj), constraints)

sol = prob.solve(solver=cp.MOSEK, verbose=False)
obj.value, v_i.value


Reassigning moment matrix indices: 100%|██████████| 9/9 [00:00<00:00, 8992.08it/s]


(-0.36356411516951304,
 array([ 4.99999983e-01, -4.69313604e-01, -5.65745067e-02,  1.74348033e-07,
        -9.31854695e-01, -1.65205475e-07,  1.02588763e+00, -8.62558911e-08,
        -5.16828804e-01]))

In [356]:
def check_if_valid_AQ_effect(effect_in_CG_raw, outcomes_per_party, settings_per_party):
    InfSDP = InflationSDP(InflationProblem(outcomes_per_party=outcomes_per_party,settings_per_party=settings_per_party))
    InfSDP.generate_relaxation(column_specification='local1',load_from_file=False)
    print(InfSDP.generating_monomials_sym)
    obj = sum([effect_in_CG_raw[i]*InfSDP.generating_monomials_sym[i] for i in range(effect_in_CG_raw.size)])
    InfSDP.set_objective()
    #InfSDP._objective_as_dict = {**{0:0, 1:1,},**{var+1:-val for var, val in enumerate(effect_in_CG_raw)}}
    InfSDP.solve(interpreter='CVXPY')
    print(InfSDP.primal_objective)
    print(InfSDP._objective_as_dict)

check_if_valid_AQ_effect(v_i.value, outcomes_per_party, settings_per_party)

Reassigning moment matrix indices: 100%|██████████| 9/9 [00:00<?, ?it/s]

[1, A_1_0_0, A_1_1_0, B_1_0_0, B_1_1_0, A_1_0_0*B_1_0_0, A_1_0_0*B_1_1_0, A_1_1_0*B_1_0_0, A_1_1_0*B_1_1_0]


TypeError: set_objective() missing 1 required positional argument: 'objective'

In [ ]:



Z = cp.Variable(positionsmatrix.shape, PSD=True)
Y = cp.Variable(positionsmatrix.shape, PSD=True)

dim = omega_alpha_c_eta_z_CG - 1
v_i = cp.Variable(dim)
v_0 = cp.Variable(1)

Fi_observable   = F_list[:dim]
Fi_unobservable = F_list[dim:]

constraints = [ *[cp.trace(F0 @ Z) == v_0],
                *[cp.trace(Fi @ Z) == v_i[i]  for i, Fi in enumerate(Fi_observable)],
                *[cp.trace(Fi @ Z) == 0       for Fi in Fi_unobservable],
                *[cp.trace(F0 @ Y) == 1-v_0],
                *[cp.trace(Fi @ Y) == -v_i[i] for i, Fi in enumerate(Fi_observable)],
                *[cp.trace(Fi @ Y) == 0    for Fi in Fi_unobservable]]obj = p_CG_CHSH[0]*v_0 + cp.sum(p_CG_CHSH[1:] @ v_i)
p_vec = np.array([1, 1/2, 1/2, 1/2, 1/2, 1/4, 1/4, 1/4, 1/4])
obj = p_vec[0]*v_0 + cp.sum(p_vec[1:] @ v_i)


prob = cp.Problem(cp.Minimize(obj),
                  constraints)

sol = prob.solve(solver=cp.MOSEK, verbose=True)
obj.value
v_0.value,v_i.value
v = np.zeros(dim+1)
v[0] = v_0.value[0]
v[1:] = v_i.value[:]
v


Reassigning moment matrix indices: 100%|██████████| 9/9 [00:00<?, ?it/s]


NameError: name 'p' is not defined

Get the omega_alpha_c_eta_z from CHSH. Get the AQ correlations that maximize CHSH.

In [ ]:
# InfProb = InflationProblem(hypergraph=[[1, 1]], outcomes_per_party=[2,2], settings_per_party=[2,2], inflation_level_per_source=[1],names=['A', 'B'])
# InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
# InfSDP.generate_relaxation( column_specification='local1',
#                             max_monomial_length=False,
#                             find_physical_monomials=False,
#                             use_linear_constraints=False,
#                             sandwich_positivity=False,
#                             load_from_file=False,
#                             filename_label=filename_label)
# meas = InfSDP.measurements
# A0, A1, B0, B1 = 1-2*meas[0][0][0][0], 1-2*meas[0][0][1][0], 1-2*meas[1][0][0][0], 1-2*meas[1][0][1][0]
# obj =  A0*B0+A0*B1+A1*B0-A1*B1
# obj = sp.expand(obj)
# print(obj)
# InfSDP.set_objective(objective=+obj)
# InfSDP.solve(interpreter='CVXPY')  # By default it maximizes
# print("minimum over AQ of CHSH", InfSDP.primal_objective)

Reassigning moment matrix indices: 100%|██████████| 9/9 [00:00<00:00, 9002.80it/s]

2 - 4*A_1_0_0 + 4*A_1_0_0*B_1_0_0 + 4*A_1_0_0*B_1_1_0 + 4*A_1_1_0*B_1_0_0 - 4*A_1_1_0*B_1_1_0 - 4*B_1_0_0
minimum over AQ of CHSH 2.8284271790023405


In [ ]:
InfProb = InflationProblem(hypergraph=[[1, 1]], outcomes_per_party=[2,2], settings_per_party=[2,2], inflation_level_per_source=[1],names=['A', 'B'])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
InfSDP.generate_relaxation( column_specification='local1',
                            max_monomial_length=False,
                            find_physical_monomials=False,
                            use_linear_constraints=False,
                            sandwich_positivity=False,
                            load_from_file=False,
                            filename_label=filename_label)
#meas = InfSDP.measurements
obj = sum([prob*sym for prob, sym in zip(p_vec, generating_set_CHSH)])
print(obj)
InfSDP.set_objective(objective=-obj)
InfSDP.solve(interpreter='CVXPY')  # By default it maximizes
print("minimum over AQ of CHSH", InfSDP.primal_objective)

In [ ]:
# def optimize_UorVorp(flag, V, U, p, InflationSDP):
#     if flag == 0: # V and p are fixed, then this is a linear problem

flag = 0
if flag == 0:  # U and p are fixed, this is then a linear program
    ...
elif flag == 1:  # V and p are fixed, this is then a linear program
    Unum = U
    shape = Unum.shape

    Uvars = cp.Variable(np.prod(shape))
    U_multidim = {}
    for idx, idxs_ in enumerate(itertools.product(*[range(i) for i in shape])):
        # idxs_ is (alpha,eta,a,b,x,y)
        U_multidim[idxs_] = Uvars[idx]  # cvxpy doesnt allow indexing of multi-dimensional variables

    constraints = [ U >= 0 ]
    for alpha,eta,x,y in itertools.product(*[range(i) for i in [*shape[:2],*shape[4:]]]):
        summ = 0
        for a,b in itertools.product(*[range(i) for i in shape[2:4]]):
            summ += U_multidim[(alpha,eta,a,b,x,y)]
        constraints += [ summ == 1 ]

    obj = 0
    for alpha,eta,a,b,x,y in itertools.product(*[range(i) for i in shape]):
        for c, z in itertools.product(*[range(i) for i in np.array(V.shape)[[1,3]].astype(int)]):
            obj += V[(alpha,c,eta,z)]*U_multidim[(alpha,eta,a,b,x,y)]*p[(a,b,c,x,y,z)]
elif flag == 2:  # V and U are fixed, then this is an SDP
    ...

# With extra output

Now we do $V_2$ such that the extra output has zero support

In [ ]:
# V2 = np.zeros((3,2,2,2))
# V2[:-1,:,:,:] = V

In [ ]:
InfProb = InflationProblem( hypergraph=[[1, 1]],
                            outcomes_per_party=[3,2],
                            settings_per_party=[2,2],
                            inflation_level_per_source=[1],
                            names=['A', 'B'])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
filename_label = 'AQ_U'
InfSDP.generate_relaxation( column_specification='local1',
                            max_monomial_length=False,
                            find_physical_monomials=False,
                            use_linear_constraints=False,
                            sandwich_positivity=False,
                            load_from_file=False,
                            filename_label=filename_label)
meas = InfSDP.measurements
# for eta, alpha in itertools.product(range(2), repeat=2):
obj = 0
#for a,b,x,y in itertools.product(*[range(i) for i in V2.shape]):
for a,b,x,y in itertools.product(*[range(i) for i in V.shape]):
    #print(a,b,x,y)
    Pi_A = 1-(meas[0][0][x][0]+meas[0][0][x][1]) if a==2 else meas[0][0][x][a] 
    Pi_B = 1-(meas[1][0][y][0]) if b==1 else meas[1][0][y][b] 
    #obj += V2[a,b,x,y]*Pi_A*Pi_B
    obj += V[a,b,x,y]*Pi_A*Pi_B
# if (alpha, eta) == (0, 1):
#     obj -= 0.8

obj = sp.expand(obj)
print("V2=", obj)
InfSDP.set_objective(objective=-obj)
InfSDP.verbose = 0
InfSDP.solve(interpreter='MOSEKFusion')  # By default it maximizes
minimo = InfSDP.primal_objective
print("minimum over AQ of V2", InfSDP.primal_objective)
InfSDP.set_objective(objective=obj)
InfSDP.verbose = 0
InfSDP.solve(interpreter='MOSEKFusion')  # By default it maximizes
print("Maximum over AQ of V2", InfSDP.primal_objective)
maximo = InfSDP.primal_objective
# Add the minimum and divide by the difference between the maximum and minimum

# print("Renormalize")
# V2[:,:,0,0] += minimo
# V2[:,:,:,:] /= maximo - minimo

# obj = 0
# for a,b,x,y in itertools.product(*[range(i) for i in V2.shape]):
#     #print(a,b,x,y)
#     Pi_A = 1-(meas[0][0][x][0]+meas[0][0][x][1]) if a==2 else meas[0][0][x][a] 
#     Pi_B = 1-(meas[1][0][y][0]) if b==1 else meas[1][0][y][b] 
#     obj += V2[a,b,x,y]*Pi_A*Pi_B
# obj = sp.expand(obj)
# print("V2 =", obj)
# InfSDP.set_objective(objective=-obj)
# InfSDP.solve()  # By default it maximizes
# minimo = InfSDP.primal_objective
# print("minimum over AQ of V2", -InfSDP.primal_objective)
# InfSDP.set_objective(objective=obj)
# InfSDP.solve()  # By default it maximizes
# print("Maximum over AQ of V2", InfSDP.primal_objective)

Reassigning moment matrix indices: 100%|██████████| 15/15 [00:00<?, ?it/s]


V2= 0.00459999999999999*A_1_0_0 + 0.224*A_1_0_0*B_1_0_0 + 0.5578*A_1_0_0*B_1_1_0 + 0.159*A_1_0_1 + 0.8372*A_1_0_1*B_1_0_0 + 0.0031*A_1_0_1*B_1_1_0 + 0.5884*A_1_1_0 - 0.5902*A_1_1_0*B_1_0_0 - 0.7404*A_1_1_0*B_1_1_0
minimum over AQ of V2 0.7422000004622425
Maximum over AQ of V2 0.9999673697516185
